[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/TotoroUI-jupyter/blob/main/ipadapter_simple.ipynb)

In [ ]:
!pip install -q torchsde einops diffusers accelerate xformers==0.0.25
!apt -y install -qq aria2

%cd /content
!git clone -b totoro https://github.com/camenduru/ComfyUI /content/TotoroUI
!git clone -b totoro https://github.com/camenduru/ComfyUI_IPAdapter_plus /content/TotoroUI/IPAdapter

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/128713 -d /content/TotoroUI/models -o dreamshaper_8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -d /content/TotoroUI/models/clip_vision -o CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -d /content/TotoroUI/models/ipadapter -o ip-adapter-plus_sd15.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/8744d2de-e74b-4b68-b46d-e3a8bd649cdf -d /content -o ip1.png
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/camenduru/TotoroUI-jupyter/assets/54370274/2fce96a5-2d85-42fc-8841-7375323149e5 -d /content -o mask.png

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/Ttl/ComfyUi_NNLatentUpscale/raw/master/sd15_resizer.pt -d /content/TotoroUI/models -o sd15_resizer.pt

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/Ttl/ComfyUi_NNLatentUpscale/raw/master/sd15_resizer.pt -d /content/TotoroUI/models -o sd15_resizer.pt

%cd /content/TotoroUI
import torch
import numpy as np
from PIL import Image
import totoro
import nodes
import sys
sys.path.append('/content/TotoroUI/IPAdapter')
import IPAdapterPlus
import scipy

with torch.no_grad():
  model_patcher, clip, vae, clipvision = totoro.sd.load_checkpoint_guess_config("/content/TotoroUI/models/dreamshaper_8.safetensors", output_vae=True, output_clip=True, embedding_directory=None)

IPAdapterPlus_model = IPAdapterPlus.IPAdapterUnifiedLoader().load_models(model_patcher, 'PLUS (high strength)', lora_strength=0.0, provider="CPU", ipadapter=None)


In [ ]:
output_image, output_mask = nodes.LoadImage().load_image("/content/ip1.png")
mask_image, mask_mask = nodes.LoadImage().load_image("/content/mask.png")
ip_model_patcher = IPAdapterPlus.IPAdapterSimple().apply_ipadapter(IPAdapterPlus_model[0], IPAdapterPlus_model[1], output_image, 0.7, 0.0, 1.0, 'standard', attn_mask=mask_mask)

latent = {"samples":torch.zeros([1, 4, 512 // 8, 512 // 8])}

tokens = clip.tokenize("a closeup of a woman astronaut in an enchanted forest, high quality, detailed")
cond, pooled = clip.encode_from_tokens(tokens, return_pooled=True)
cond = [[cond, {"pooled_output": pooled}]]
n_tokens = clip.tokenize("blurry, lowres, bad art, ill, distorted, malformed, horror")
n_cond, n_pooled = clip.encode_from_tokens(n_tokens, return_pooled=True)
n_cond = [[n_cond, {"pooled_output": n_pooled}]]

sample = nodes.common_ksampler(model=ip_model_patcher[0], 
                          seed=4, 
                          steps=30, 
                          cfg=7.0, 
                          sampler_name="dpmpp_2m", 
                          scheduler="karras", 
                          positive=cond, 
                          negative=n_cond,
                          latent=latent, 
                          denoise=1)

with torch.inference_mode():
  sample = sample[0]["samples"].to(torch.float16)
  vae.first_stage_model.cuda()
  decoded = vae.decode_tiled(sample).detach()

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])

In [ ]:
from latent_resizer import LatentResizer
from totoro import model_management

def upscale(latent, upscale):
    device = model_management.get_torch_device()
    samples = latent.to(device=device, dtype=torch.float16)
    model = LatentResizer.load_model('/content/TotoroUI/models/sd15_resizer.pt', device, torch.float16)
    model.to(device=device)
    latent_out = (model(0.13025 * samples, scale=upscale) / 0.13025)
    latent_out = latent_out.to(device="cpu")
    model.to(device=model_management.vae_offload_device())
    return ({"samples": latent_out},)

latent = upscale(sample, 1.5)

sample = nodes.common_ksampler(model=ip_model_patcher[0], 
                          seed=543543, 
                          steps=30, 
                          cfg=7.0, 
                          sampler_name="dpmpp_2m", 
                          scheduler="karras", 
                          positive=cond, 
                          negative=n_cond,
                          latent=latent[0], 
                          denoise=0.55)

with torch.inference_mode():
  sample = sample[0]["samples"].to(torch.float16)
  vae.first_stage_model.cuda()
  decoded = vae.decode_tiled(sample).detach()

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])